# Recuperação da Informação e Busca na Web - 2018.1

### Atividade: Lab 02 - Parte 1 - Expansão de Consultas
### Aluno: Johanny de Lucena Santos

## - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Nesta atividade você vai exercitar a noção de expansão de consultas. Considerando a coleção de notícias do lab passado, você primeiro precisa executar os seguintes passos:

1. Escreva uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).
2. Escreva uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.
3. Expanda a consulta original com os termos retornados no passo 2 acima.
4. Faça uma busca disjuntiva (OR) considerando a nova consulta.

## Importações necessárias para a análise dos dados

In [1]:
import pandas as pd

import numpy as np

from scipy import sparse

import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from collections import Counter

In [102]:
#nltk.download('stopwords')



## Definindo o arquivo com os dados a ser analisado
Esse é o arquivo base de estudo, que contém notícias de um determinado período de tempo relacionados à eleições, da base de notícias do Estadão.

Primeiramente, fazemos a importação do arquivo no formato .csv para tratamento, e em seguida, removemos as linhas com valores NaN (not a number) para não atrapalhar nas operações

In [2]:
dataset = pd.read_csv('../data/estadao_noticias_eleicao.csv', encoding="utf-8")
dataset = dataset.replace(np.nan, '', regex=True)

## Join dos conteúdos

Realizando junção dos títulos, subtítulos e conteúdos

In [3]:
documentos = dataset.titulo + " " + dataset.subTitulo + " " + dataset.conteudo
documentos = documentos.fillna("")

## Criação dos docIDs

In [4]:
docIDs = dataset.idNoticia

## Função que gera uma matriz de co-occorrência e vocabulário

Essa função recebe o corpus (documentos) e retorna uma matriz termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas), bem como o vocabulário.

In [5]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

## Removendo pontuação
As pontuações estão sendo removidas para que a tokenização seja feita apenas com as palavras

In [6]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = documentos.apply(lambda text: tokenizer.tokenize(text.lower()))

## Removendo stopwords
Removendo as palavras que podem ser consideradas irrelevantes para o conjunto de resultados a ser exibido em uma busca.

In [7]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

## Criando lista de tokens e matriz de co-ocorrência e vocabulário

In [8]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [9]:
matrix, vocab = co_occurrence_matrix(tokens)

## Frequência de termos

In [10]:
term_frequency = tokens_lists.apply(Counter)

## Gerando índice invertido

In [11]:
indiceInvertido = {}

def gerarIndiceInvertido():
    for i in range(len(tokens_lists)):
        idNoticia = docIDs[i]
        palavras = tokens_lists[i]
    
        for palavra in palavras:
            if palavra not in indiceInvertido:
                indiceInvertido[palavra.lower()] = {}
        
            if not indiceInvertido[palavra.lower()].get(idNoticia):
                docs = indiceInvertido[palavra.lower()]
                docs[idNoticia] = term_frequency[i][palavra.lower()]

In [12]:
gerarIndiceInvertido()

## Função para gerar um dicionário com os pesos dos índices

In [13]:
def gerar_dict_pesos(termos, gerador_peso):

    docs_peso = {}
    
    for i in range(len(termos)):
        termo = termos[i].lower()
        docs = indiceInvertido[termo]
        
        for doc_id in docs:
            tf = docs[doc_id]
            
            if doc_id not in docs_peso:
                docs_peso[doc_id] = np.array([0 if j != i else gerador_peso(tf) for j in range(len(termos))])
            else:
                doc_vector = docs_peso[doc_id]
                docs_peso[doc_id] = np.array([doc_vector[j] if j != i else gerador_peso(tf) for j in range(len(termos))])
    
    return docs_peso

## Função para gerar o vetor de consulta

In [14]:
def gerar_consulta(termos):
    vetor = np.array([1 if indiceInvertido.get(termo.lower()) else 0 for termo in termos])
    return vetor

## Função para gerar o TF da consulta

In [15]:
def gerar_vetor_tf(frase):
    def gerador_peso(tf):
        return tf
    return gerar_dict_pesos(frase, gerador_peso)

## Função de busca por TF

In [16]:
def busca_tf(frase):
    docs_tf = gerar_vetor_tf(frase)
    query = gerar_consulta(frase)
    doc_rank = sorted(list(docs_tf.items()), key=lambda doc: np.dot(doc[1], query), reverse=True)
    return [doc[0] for doc in doc_rank]

## Função de busca disjuntiva (OR)

In [17]:
def busca_or(*termos):
    uniao = set(indiceInvertido[termos[0].lower()])
    for termo in termos:
        ids = set(indiceInvertido[termo.lower()])
        uniao = uniao.union(ids)
    return list(uniao)

## Criação de uma matriz consultável com base na matriz de co-ocorrência

In [18]:
consultable_matrix = matrix.tocsr()

## Função que retorna a frequência entre duas palavras
Retorna a frequência entre duas palavras

In [19]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
## Função que retorna as top 3 palavras da consulta

Esta função recebe um certo termo para consultar com base na matriz construída (consultable_matrix) anteriormente e retorneas top-3 palavras em ordem decrescente de frequência.

In [20]:
def top_3_words(termo):
    ranking = consultable_matrix[vocab[termo]].getrow(0).toarray()[0]
    indices, freq = zip(*sorted(enumerate(ranking), key=lambda x: x[1], reverse=True))
    
    return indices[:3], ranking[:3]

In [21]:
def top_3_words_txt(termo):
    ranking = consultable_matrix[vocab[termo]].getrow(0).toarray()[0]
    indices, freq = zip(*sorted(enumerate(ranking), key=lambda x: x[1], reverse=True))
    
    return ([p for key in indices[:3] for p in vocab.keys() if vocab[p] == key])

## Função de busca expandida
Expansão da consulta original com os termos retornados na função **top_3_words**

In [84]:
def busca_expandida(termo):
    expansao = top_3_words_txt(termo)
    expansao.append(termo)
    doc_expandido = busca_tf(expansao)
    return (expansao, doc_expandido)

## Função para imprimir os resultados das buscas expandidas e original

In [80]:
def imprime_buscas(termo):
    expansao, doc_expandido = busca_expandida(termo)
    busca_original = busca_tf([termo])
    print("Termo: '", termo, "'")
    print("Expansao: ",expansao)
    print("Busca expandida: ", len(doc_expandido))
    print("Busca original: ", len(busca_original))
    print(" ")

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

## Expansão da busca

In [87]:
imprime_buscas("dilma")

imprime_buscas("michel")

imprime_buscas("impeachment")

print("Número de documentos na busca disjuntiva: ", len(busca_or("dilma", "michel", "impeachment")))

Termo: ' dilma '
Expansao:  ['rousseff', 'disse', 'é', 'dilma']
Busca expandida:  7397
Busca original:  4270
 
Termo: ' michel '
Expansao:  ['temer', 'saliba', 'louis', 'michel']
Busca expandida:  418
Busca original:  334
 
Termo: ' impeachment '
Expansao:  ['presidente', 'dilma', 'fernando', 'impeachment']
Busca expandida:  6141
Busca original:  96
 
Número de documentos na busca disjuntiva:  4361


## Perguntas

### 1. Quais os termos retornados para a expansão de cada consulta?

**"Dilma":** rousseff; disse; é;

**"Michel":** temer; saliba; louis;

**"Impeachment":** presidente; dilma; fernando

### 2. Você acha que esses termos são de fato relacionados com a consulta original? Justifique.

Sim, pois as top-3 palavras que mais aparecem nas consultas estão relacionadas com o termo. Por exemplo, na busca por "Dilma" a palavra mais frequente é o seu sobrenome, "Rousseff"; o mesmo acontece na consulta por "Impeachment", onde são retornadas as palavras "presidente", "Dilma", e "Fernando", referenciando que os dois nomes citados, Dilma Rousseff e Fernando Color, estão ambos associados com "impeachment".

### 3. Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?

Acredito que a busca expandida melhor captura a necessidade de informação do usuário, pois ela retorna muito mais documentos do que a busa original. Além do mais, a busca expandida retorna os resultados que estão mais relacionados com o termo pesquisado.

### 4. A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?
<p>Recall (Revocação): é a fração de instâncias relevantes que são recuperadas.</p>
<p>Precision (Precisão): é a fração de instâncias recuperadas que são relevantes</p>
![Recall](https://wikimedia.org/api/rest_v1/media/math/render/svg/00783a9f77bb578eb51426da033d3cdc62cc0ea4)

![Precision](https://wikimedia.org/api/rest_v1/media/math/render/svg/07edd83970d9cb357f76ba98166fe07cabc2d385)

Mais adequada para melhorar o recall, pois o número de documentos relevantes recuperados será maior, dado o número de documentos relevantes. Quanto maior a quantidade de documentos relevantes recuperados, maior será a quantidade de documentos relevantes contidos na busca. Entretanto, o precision será menor, já que ele é definido pela razão entre a quantidade de documentos relevantes na busca e a quantidade de documentos relevantes recuperados.